# setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver

# tools

In [4]:
search_tool = TavilySearch(max_results=2)
tools = [search_tool]

# graph

In [9]:
from typing import Annotated

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

llm_with_tools = llm.bind_tools(tools)
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    return {"messages": [message]}

In [21]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# multi-step usage with resuming example

In [22]:
config = {"configurable": {"thread_id": "1122"}}

In [23]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_3LXyg3h9D4lJUgEFtmZtwQSo)
 Call ID: call_3LXyg3h9D4lJUgEFtmZtwQSo
  Args:
    query: LangGraph
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. It provides a set of tools and libraries that enable users to create, run and optimize large language models (LLMs) in a scalable and efficient manner. At its core, LangGr

In [24]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                    "Can you give me some resources with explaination"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!Can you give me some resources with explaination
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_LmsywJCwvGRZPnntIN4Wvee6)
 Call ID: call_LmsywJCwvGRZPnntIN4Wvee6
  Args:
    query: LangGraph autonomous agent tutorial
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph autonomous agent tutorial", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.projectpro.io/article/langgraph/1109", "title": "LangGraph Tutorial for Beginners to Build AI Agents - ProjectPro", "content": "Last Updated: 12 Mar 2025  | BY ProjectPro\nIf you're eager to build intelligent, multi-agent AI systems that don’t just react but remember, adapt, and collaborate, LangGr

In [27]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  8 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
-----------------------------------------

In [28]:
print(to_replay.next)
print(to_replay.config)

('tools',)
{'configurable': {'thread_id': '1122', 'checkpoint_ns': '', 'checkpoint_id': '1f033d9e-252b-629c-8006-9708e4d2bb09'}}


In [29]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_LmsywJCwvGRZPnntIN4Wvee6)
 Call ID: call_LmsywJCwvGRZPnntIN4Wvee6
  Args:
    query: LangGraph autonomous agent tutorial
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph autonomous agent tutorial", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.projectpro.io/article/langgraph/1109", "title": "LangGraph Tutorial for Beginners to Build AI Agents - ProjectPro", "content": "Last Updated: 12 Mar 2025  | BY ProjectPro\nIf you're eager to build intelligent, multi-agent AI systems that don’t just react but remember, adapt, and collaborate, LangGraph is your go-to tool. This LangGraph tutorial will walk you through its core concepts, helping you go beyond basic AI agents to create stateful, multi-agent workflows that handle user input, to